## Imports

In [12]:
import numpy as np

## Load training data

In [17]:
X_emb = np.load("processed_train/X_emb.npy", allow_pickle=True)
Xt_emb = np.load("processed_train/Xt_emb.npy", allow_pickle=True)
yr = np.load("processed_train/yr.npy", allow_pickle=True)
ys =np.load("processed_train/ys.npy", allow_pickle=True)
yr = yr.astype('float64') 
ys = ys.astype('float64') 

y = np.concatenate((np.transpose([yr]), np.transpose([ys])), axis=1)

print(f"type(X_emb)-->{type(X_emb)}")
print(f"type(Xt_emb)-->{type(Xt_emb)}")
print(f"type(ys)-->{type(ys)}")
print(f"type(yr)-->{type(yr)}\n")

print(f"y.shape-->{y.shape}")
print(f"type(y)-->{type(y)}\n")

print(f"X_emb.shape-->{X_emb.shape}")
print(f"Xt_emb.shape-->{Xt_emb.shape}")
print(f"ys.shape-->{ys.shape}")
print(f"yr.shape-->{yr.shape}")

type(X_emb)--><class 'numpy.ndarray'>
type(Xt_emb)--><class 'numpy.ndarray'>
type(ys)--><class 'numpy.ndarray'>
type(yr)--><class 'numpy.ndarray'>

y.shape-->(1400, 2)
type(y)--><class 'numpy.ndarray'>

X_emb.shape-->(1400, 864, 300)
Xt_emb.shape-->(200, 864, 300)
ys.shape-->(1400,)
yr.shape-->(1400,)


## LSTM

In [18]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Bidirectional

import keras.backend as K

# model initialization
model = Sequential()
model.add(Bidirectional(LSTM(64), input_shape = (X_emb.shape[1], X_emb.shape[2])))
model.add(Dense(24, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
#model.build(input_shape = (64, X_emb.shape[1], X_emb.shape[2]))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_6 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dense_12 (Dense)            (None, 24)                3096      
                                                                 
 dense_13 (Dense)            (None, 2)                 50        
                                                                 
Total params: 190,026
Trainable params: 190,026
Non-trainable params: 0
_________________________________________________________________


In [19]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=5, patience=4)

model.fit(X_emb, y, epochs=50, batch_size=32, validation_split=0.1, callbacks=[es] )

Epoch 1/50
40/40 [==============================] - 71s 2s/step - loss: 0.6533 - accuracy: 0.5071 - val_loss: 0.6241 - val_accuracy: 0.5643
Epoch 2/50
40/40 [==============================] - 64s 2s/step - loss: 0.5095 - accuracy: 0.6683 - val_loss: 0.5304 - val_accuracy: 0.8500
Epoch 3/50
40/40 [==============================] - 53s 1s/step - loss: 0.4736 - accuracy: 0.6865 - val_loss: 0.4690 - val_accuracy: 0.7571
Epoch 4/50
40/40 [==============================] - 58s 1s/step - loss: 0.4533 - accuracy: 0.7183 - val_loss: 0.4326 - val_accuracy: 0.5786
Epoch 5/50
40/40 [==============================] - 56s 1s/step - loss: 0.3631 - accuracy: 0.7286 - val_loss: 0.3825 - val_accuracy: 0.7214
Epoch 6/50
40/40 [==============================] - 45s 1s/step - loss: 0.3406 - accuracy: 0.7437 - val_loss: 0.3512 - val_accuracy: 0.7000
Epoch 7/50
40/40 [==============================] - 44s 1s/step - loss: 0.4564 - accuracy: 0.7048 - val_loss: 0.5357 - val_accuracy: 0.6000
Epoch 8/50
40/40 [==